In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from stockpy.neural_network import MLPClassifier, CNNClassifier, LSTMClassifier
from stockpy.probabilistic import BayesianNNClassifier, BayesianCNNClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
data = pd.read_pickle('../../test/data.pickle')
X = data.drop(['scenario'], axis=1)
y = data['scenario']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# create model instance and fit to training data
predictor = BayesianCNNClassifier(hidden_size=32)
predictor.fit(X=X_train, 
              y=y_train,
              patience=5,
              batch_size=24,
              sequence_length=22,
              pretrained=False,
              scaler_type='minmax',
              epochs=50)

# Append true labels and predicted labels to lists using the score method
true_labels, pred_labels = predictor.score(X_test, y_test)

In [ ]:
# print confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
# plot the confusion matrix
plt.figure(figsize=(8,8))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix', size=15)
plt.show()

In [ ]:
def plot_training_metrics(train_losses, train_accuracies, train_f1_scores):
    epochs = range(1, len(train_losses) + 1)
    sns.set(style="ticks", context="talk")
    plt.style.use('dark_background')
    file = file = '../results/classifier' + predictor.__class__.__name__ + '.png'
    fig, ax = plt.subplots(3, 1, figsize=(12, 18), sharex=True)

    ax[0].plot(epochs, train_losses, label='Training Loss', linewidth=3, color='#2962FF')
    ax[1].plot(epochs, train_accuracies, label='Training Accuracy', linewidth=3, color='#FFA726')
    ax[2].plot(epochs, train_f1_scores, label='Training F1 Score', linewidth=3, color='#4CAF50')

    ax[0].set_title('Training Loss', fontsize=24, color='white')
    ax[1].set_title('Training Accuracy', fontsize=24, color='white')
    ax[2].set_title('Training F1 Score', fontsize=24, color='white')

    for i in range(3):
        ax[i].set_xlabel('Epoch', fontsize=16, color='white', labelpad=10)
        ax[i].set_ylabel('Value', fontsize=16, color='white', labelpad=10)
        ax[i].grid(color='white', linestyle='--', linewidth=0.5)
        ax[i].legend(fontsize=16, loc='upper right', edgecolor='black')

    plt.tight_layout()
    plt.savefig(file, dpi=80, facecolor='#131722')
    plt.show()